In [ ]:
# https://www.rieselprime.de/ziki/Self-initializing_quadratic_sieve
N = 11952495694293596231 * 73504159268087930309 

In [ ]:
from sage.misc.prandom import randint

class Factor_Base:
    def __init__(self, p, tsqrt, tlog):
        self.p = p
        self.sol_1 = None
        self.sol_2 = None
        self.tsqrt = tsqrt
        self.tlog = tlog
        self.ainv = None
        
class Polynomial:
    def __init__(self, coef=[], a=None, b=None):
        self.a = a
        self.b = b
        self.coef = coef

    def calc(self, x):
        res = 0
        for a in self.coef[::-1]:
            res *= x
            res += a
        return res

In [ ]:
def choose_params(dig):
    if dig <= 34:
        return 200, 65536
    if dig <= 36:
        return 300, 65536
    if dig <= 38:
        return 400, 65536
    if dig <= 40:
        return 500, 65536
    if dig <= 42:
        return 600, 65536
    if dig <= 44:
        return 700, 65536
    if dig <= 48:
        return 1000, 65536
    if dig <= 52:
        return 1200, 65536
    if dig <= 56:
        return 2000, 65536 * 3
    if dig <= 60:
        return 4000, 65536 * 3
    if dig <= 66:
        return 6000, 65536 * 3
    if dig <= 74:
        return 10000, 65536 * 3
    if dig <= 80:
        return 30000, 65536 * 3
    if dig <= 88:
        return 50000, 65536 * 3
    if dig <= 94:
        return 60000, 65536 * 9
    return 100000, 65536 * 9

def sqrt_mod(a, p):
    # квадратный корень по модулю, mod(a,b).sqrt() работает как-то не так
    assert a < p
    assert is_prime(p)
    if a == 0:
        return 0
    if p == 2:
        return a
    if p % 2 == 0:
        return None
    p_mod_8 = p % 8
    if p_mod_8 == 1:
        # Алгоритм Шенкса (https://ru.bmstu.wiki/%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC_%D0%A8%D0%B5%D0%BD%D0%BA%D1%81%D0%B0)
        q = p // 8
        e = 3
        while q % 2 == 0:
            q //= 2
            e += 1
        while True:
            x = randint(2, p - 1)
            z = mod(x^q,p)
            if mod(z^(2 ^ (e - 1)), p) != 1:
                break
        y = z
        r = e
        x = mod(a ^ ((q - 1) // 2), p)
        v = (a * x) % p
        w = (v * x) % p
        while True:
            if w == 1:
                return v
            k = 1
            while mod(w ^ (2 ^ k), p) != 1:
                k += 1
            d = mod(y ^ (2 ^ (r - k - 1)), p)
            y = (d ^ 2) % p
            r = k
            v = (d * v) % p
            w = (w * y) % p
    elif p_mod_8 == 5:
        v = mod((2 * a) ^ ((p - 5) // 8), p)
        i = (2 * a * v * v) % p
        return (a * v * (i - 1)) % p
    else:
        return mod(a ^ ((p + 1) // 4), p)

def gen_factor_base(N, nf):
    factor_base = []
    count = 0
    p = 2
    #print('fac:', p)
    tsqrt = sqrt_mod(N % p ,p)
    tlog = log(p,2).round()
    # легче хранить массив таких структур чем несколько массивов
    factor_base.append(Factor_Base(p,tsqrt,tlog))
    count += 1
    while True:
        p = p.next_prime()
        if kronecker(N, p) == 1:
            #print('fac:', p)
            tsqrt = sqrt_mod(N % p ,p)
            tlog = log(p,2).round()
            
            factor_base.append(Factor_Base(p,tsqrt,tlog))
            count += 1
            #print('fac:', (p, tsqrt,tlog))
            if count == nf:
                break
    
    return(factor_base)

def first_poly(N, m, factor_base):
    # Сгенерировать первый полином для самоинициализирующегося мультиполиномиального квадратичного решета
    min_i = None
    max_i = None
    for i, factor in enumerate(factor_base):
        if min_i is None and factor.p >= 400:
            min_i = i
        if max_i is None and factor.p > 4000:
            max_i = i - 1
            break

    # при маленькой факторной базе
    if max_i is None:
        max_i = len(factor_base) - 1
    if min_i is None or max_i - min_i < 20:
        min_i = min(min_i, 5)
    
    #print('p_i:',min_i, max_i)
    #print(factor_base[min_i].p, factor_base[max_i].p)
    near = sqrt(2*float(N))/m
    near_2 = float(near / sqrt((factor_base[min_i].p + factor_base[max_i].p) / 2))
    #print(near, near_2)
    best_q, best_a, best_ratio = None, None, None
    for _ in range(30): 
        a = 1
        q = []

        while a < near_2:
            p_i = 0
            while p_i == 0 or p_i in q:
                p_i = randint(min_i, max_i)
            p = factor_base[p_i].p
            a *= p
            q.append(p_i)

        ratio = a / near

        if (best_ratio is None or (ratio >= 0.9 and ratio < best_ratio) or
                    best_ratio < 0.9 and ratio > best_ratio):
            best_q = q
            best_a = a
            best_ratio = ratio
    a = best_a
    q = best_q

    s = len(q)
    B = []
    for i in range(s):
        fb_i = factor_base[q[i]]
        q_i = fb_i.p
        assert a % q_i == 0
        gamma = Integer(mod((fb_i.tsqrt * inverse_mod(a // q_i, q_i)), q_i))
        if gamma > q_i // 2:
            gamma = q_i - gamma
        B.append(a // q_i * gamma)

    b = sum(B) % a
    bb = b
    if (2 * b > a):
        b = a - b

    assert 0 < b
    assert 2 * b <= a
    #print(b, a)
    assert ((b * b - N) % a == 0)

    g = Polynomial([b^2 - N, 2 * a * b, a^2], a, bb)
    h = Polynomial([b, a])
    for factor in factor_base:
        if a % factor.p != 0:
            factor.ainv = Integer(inverse_mod(a, factor.p))
            factor.sol_1 = Integer(mod(factor.ainv * (factor.tsqrt - b), factor.p))
            factor.sol_2 = Integer(mod(factor.ainv * (-factor.tsqrt - b), factor.p))

    return g, h, B

def gen_poly(N, factor_base, i, g, B):
    # генерировать полиномы кроме первого
    # номер младшего установленного бита = log(i&-i,2) 
    v = log(i&-i,2)  + 1
    z = -1 if mod(ceil(i / (2^v)), 2) == 1 else 1
    b = Integer(mod((g.b + 2 * z * B[v - 1]), g.a))
    a = g.a
    bb = b
    if (2 * b > a):
        b = a - b
    assert ((b^2 - N) % a == 0)

    g = Polynomial([b^2 - N, 2*a*b, a^2], a, bb)
    h = Polynomial([b, a])
    for factor in factor_base:
        if a % factor.p != 0:
            factor.sol_1 = Integer(mod(factor.ainv * (factor.tsqrt - b), factor.p))
            factor.sol_2 = Integer(mod(factor.ainv * (-factor.tsqrt - b), factor.p))

    return g, h

def gen_array(factor_base, m):
    # Создать массив для просеивания
    sieving_array = [0] * (2 * m + 1)
    for factor in factor_base:
        if factor.sol_1 is None:
            continue
        p = factor.p
        #print('pmfactorsol1:', p, m, factor.sol_1)
        #print('types', type(p), type(m), type(factor.sol_1))
        i_start_1 = -((m + factor.sol_1) // p)
        a_start_1 = factor.sol_1 + i_start_1 * p
        tlog = factor.tlog
        if p > 20:
            for a in range(a_start_1 + m, 2 * m + 1, p):
                sieving_array[a] += tlog

            i_start_2 = -((m + factor.sol_2) // p)
            a_start_2 = factor.sol_2 + i_start_2 * p
            for a in range(a_start_2 + m, 2 * m + 1, p):
                sieving_array[a] += tlog
    return sieving_array

def get_multiples(a, factor_base):
    # возвращает список кортежей с элементами из факторной базы, которые делят a
    divisor_list = []
    for i, factor in enumerate(factor_base):
        if mod(a, factor.p) == 0:
            power = 0
            while mod(a, factor.p) == 0:
                a //= factor.p
                power += 1
            divisor_list.append((i, power))
        if a == 1:
            return divisor_list
    return None

def trial_division(N, sieving_array, factor_base, smooth, g, h, m,
                        req):
    # перебор делителей
    sqrt_n = n(sqrt(N))
    
    limit = log(m*sqrt_n, 2) - 25
    for (i, j) in enumerate(sieving_array):
        if j >= limit:
            x = i - m
            g_x = g.calc(x)
            divisor_list = get_multiples(g_x, factor_base)
            if divisor_list is not None:
                u = h.calc(x)
                v = g_x
                assert mod((u^2), N) == mod(v, N)
                smooth.append((u, v, divisor_list))
                if (len(smooth) >= req):
                    return True
    return False

def create_matrix(factor_base, smooth):
    # построение матрицы
    flen = len(factor_base)
    M = []
    for rel in smooth:
        mi = [0] * flen
        for j, exp in rel[2]:
            mi[j] = exp % 2
        M.append(mi)
    return M


def convert_matrix(M):
    m = len(M[0])
    cols_binary = [""] * m
    for mi in M:
        for j, mij in enumerate(mi):
            cols_binary[j] += "1" if mij else "0"
    return [int(cols_bin[::-1], 2) for cols_bin in cols_binary], len(M), m

def find_col(M_conv, j):
    # Найти строку с первым ненулевым вхождение в столбце j
    if M_conv[j] == 0:
        return None
    return log(M_conv[j]&-M_conv[j],2) 


def matrix_solving(M_conv, N, m):
    row_flag = [False] * N
    pivots = [-1] * m
    for j in range(m):
        i = find_col(M_conv, j)
        if i is not None:
            pivots[j] = i
            row_flag[i] = True
            for k in range(m):
                if k != j and (M_conv[k] >> i) & 1:
                    M_conv[k] ^^= M_conv[j]
    perf_sq = []
    for i in range(N):
        if not row_flag[i]:
            perf_sq_i = [i]
            for j in range(m):
                if (M_conv[j] >> i) & 1:
                    perf_sq_i.append(pivots[j])
            perf_sq.append(perf_sq_i)
    return perf_sq


def calc_ab(sq_i, smooth):
    # Найти на основе matrix_solving и smooth a,b такие, что a^2 = b^2 (mod N)
    res = [1, 1]
    for i in sq_i:
        res[0] *= smooth[i][0]
        res[1] *= smooth[i][1]
    res[1] = floor(sqrt(res[1]))
    return res

def ret_fact(N, sq_i, smooth):
    sqrt1, sqrt2 = calc_ab(sq_i, smooth)
    assert mod(sqrt1^2, N) == mod(sqrt2^2, N)
    return gcd(abs(sqrt1 - sqrt2), N)

def more_fact(numbers):
    res = set()
    for num in numbers:
        res.add(num)
        for m in numbers:
            if num != m:
                fact = gcd(num, m)
                if fact != 1 and fact != num and fact != m:
                    if fact not in res:
                        print(f"Find more: {fact}")
                        res.add(fact)
                    res.add(num // fact)
                    res.add(m // fact)
    return res

def give_factors(N, perfect_squares, smooth):
    factors = []
    rem = N
    np_factors = set()
    p_factors = set()
    for sq_i in perfect_squares:
        fact = ret_fact(N, sq_i, smooth)
        if fact != 1 and fact != rem:
            if is_prime(fact):
                if fact not in p_factors:
                    print (f"p_factor found: {fact}")
                    p_factors.add(fact)

                while rem % fact == 0:
                    factors.append(fact)
                    rem //= fact

                if rem == 1:
                    break
                if is_prime(rem):
                    factors.append(rem)
                    rem = 1
                    break
            else:
                if fact not in np_factors:
                    print (f"np_factor found: {fact}")
                    np_factors.add(fact)

    if rem != 1 and np_factors:
        np_factors.add(rem)
        for fact in sorted(more_fact(np_factors)):
            while fact != 1 and rem % fact == 0:
                print (f"factor found: {fact}")
                factors.append(fact)
                rem //= fact
            if rem == 1 or is_prime(rem):
                break

    if rem != 1:
        factors.append(rem)
    return factors

In [ ]:
dig = len(str(N))
nf, m = choose_params(dig)
factor_base = gen_factor_base(N,nf)
ratio = 1.05
flag = False
smooth = []
prev_cnt = 0
i_poly = 0
while not flag:
    req = round(len(factor_base) * ratio)
    enough_relations = False
    while not enough_relations:
        if i_poly == 0:
            g, h, B = first_poly(N, m, factor_base)
        else:
            g, h = gen_poly(N, factor_base, i_poly, g, B)
        i_poly += 1
        if i_poly >= 2^(len(B) - 1):
            i_poly = 0
        sieve_array = gen_array(factor_base, m)
        
        enough_relations = trial_division(N, sieve_array, factor_base, smooth, g, h, m, req)

        if (len(smooth) >= req or i_poly % 8 == 0 and len(smooth) > prev_cnt):
            print(f"{len(smooth)} from {req}")
            prev_cnt = len(smooth)

    print("Create matrix")
    M = create_matrix(factor_base, smooth)
    M_conv, M_n, M_m = convert_matrix(M)

    perfect_squares = matrix_solving(M_conv, M_n, M_m)

    print("Getting factors")
    factors = give_factors(N, perfect_squares, smooth)
    if len(factors) > 1:
        flag = True
    else:
        print("Fail, retrying...")
        ratio += 0.05

print(factors)